Universidad del valle de Guatemala  
Dpto. Ciencias de la computacion  
Inteligencia Artificial  
Alberto Suriano  

Laboratorio 8
Andres Quinto - 18288  
Marlon Hernández - 15177  

[Repositorio_aqui](https://github.com/AndresQuinto5/IA_LAB08.git)

### Tasks 1 - Teoría  

1. Investigar el **algoritmo AC-3** y su relación con el algoritmo de **backtracking search**  
    El **algoritmo AC-3** es un algoritmo de consistencia de arcos utilizado en problemas de satisfacción de restricciones (CSP). Su objetivo es reducir los dominios de las variables eliminando valores que no tienen soporte, es decir, que no pueden formar parte de una solución consistente con las restricciones. Por otro lado, **el algoritmo de backtracking search** es una técnica de búsqueda que intenta construir una solución paso a paso, retrocediendo cuando encuentra que una asignación de valores no lleva a una solución válida. La relación entre ambos es que **AC-3** puede utilizarse antes del **backtracking** para preprocesar el CSP, reduciendo los dominios y, por lo tanto, el número de asignaciones a probar durante el backtracking  

    Un ejemplo de este algoritmo podria ser, tenes una caja de lápices de colores y quieres asegurarte de que puedes dibujar un arcoíris completo. Pero hay una regla: cada color solo puede ir al lado de ciertos colores. **El algoritmo AC-3** es como un amigo que revisa todos los lápices y se asegura de que cada uno tenga un vecino adecuado antes de empezar a dibujar. Así, cuando comiences a colorear, no te detendrás a mitad de camino porque todos los lápices están en el orden correcto para hacer un arcoíris perfecto.

    **referencias:**
    - [Algoritmo AC-3](https://en.wikipedia.org/wiki/AC-3_algorithm)
    - [Backtracking Algorithms](https://www.freecodecamp.org/news/backtracking-algorithms-recursive-search/)

    
2. Defina en sus propias palabras el término “Arc Consistency”  
    **En mis propias palabras, “Arc Consistency”** se refiere a un estado en el que, para cada par de variables en un CSP que comparten una restricción, cada valor de la primera variable tiene al menos un valor correspondiente en la segunda variable que satisface la restricción entre ellas. Esto asegura que no hay valores aislados que hagan imposible encontrar una solución completa al problema.

    Un ejemplo podria un juego de parejas de cartas. Cada carta tiene un número y debes encontrarle una pareja que tenga el mismo número. **“Arc Consistency”** significa que todas las cartas tienen al menos una pareja posible. Si alguna carta no tuviera pareja, no podrías ganar el juego. Entonces, antes de jugar, revisas todas las cartas para asegurarte de que cada una tiene una pareja y así sabes que el juego se puede ganar.

    **referencias:**
    - [Arc Consistency](https://en.wikipedia.org/wiki/Arc_consistency)
    - [Arc Consistency Explained](https://www.boristhebrave.com/2021/08/30/arc-consistency-explained/)

### Task 2 - CSP con Backtracking, Beam y Local Search

In [8]:
# Definir variables (exámenes/cursos)
exams = ['Exam1', 'Exam2', 'Exam3', 'Exam4', 'Exam5', 'Exam6', 'Exam7']

# Definir dominio (días posibles para cada examen)
days = ['Monday', 'Tuesday', 'Wednesday']

# Definir estudiantes y los exámenes que toman
students = {
    'Student1': ['Exam1', 'Exam2', 'Exam3'],
    'Student2': ['Exam2', 'Exam4', 'Exam5'],
    'Student3': ['Exam1', 'Exam6', 'Exam7'],
    'Student4': ['Exam3', 'Exam5', 'Exam7']
}

# Función para verificar si una asignación cumple con las restricciones
def is_valid_assignment(assignment):
    # Verificar que todos los exámenes se realicen en días diferentes
    if len(set(assignment.values())) != len(assignment):
        return False
    
    # Verificar que ningún estudiante tenga más de un examen por día
    for student, student_exams in students.items():
        student_exam_days = [assignment[exam] for exam in student_exams if exam in assignment]
        if len(set(student_exam_days)) != len(student_exam_days):
            return False
    
    # Verificar que los estudiantes que toman el mismo curso no tengan exámenes el mismo día
    for exam in exams:
        exam_students = [student for student, student_exams in students.items() if exam in student_exams]
        exam_days = [assignment[exam] for student in exam_students if exam in assignment]
        if len(set(exam_days)) != len(exam_days):
            return False
    
    return True

#### Implementamos el algoritmo de backtracking search

In [9]:
import time

# Función de backtracking search
def backtracking_search(assignment={}):
    if len(assignment) == len(exams):
        return assignment
    
    unassigned_exam = [exam for exam in exams if exam not in assignment][0]
    
    for day in days:
        new_assignment = assignment.copy()
        new_assignment[unassigned_exam] = day
        
        if is_valid_assignment(new_assignment):
            result = backtracking_search(new_assignment)
            if result is not None:
                return result
    
    return None

# Ejecutar el algoritmo de backtracking search y medir el tiempo
start_time = time.time()
solution = backtracking_search()
end_time = time.time()

# Imprimir la solución y el tiempo de ejecución
print("Backtracking Search Solution:")
if solution is None:
    print("No se encontró una solución válida.")
else:
    for exam, day in solution.items():
        print(f"{exam}: {day}")
    print(f"\nTiempo de ejecución: {end_time - start_time:.5f} segundos")

Backtracking Search Solution:
No se encontró una solución válida.


#### Implementamos el algoritmo de beam search

In [10]:
import time

# Función de beam search
def beam_search(beam_width):
    initial_state = {}
    beam = [(initial_state, 0)]
    
    while beam:
        next_beam = []
        
        for state, cost in beam:
            if len(state) == len(exams):
                return state
            
            unassigned_exam = [exam for exam in exams if exam not in state][0]
            
            for day in days:
                new_state = state.copy()
                new_state[unassigned_exam] = day
                
                if is_valid_assignment(new_state):
                    next_beam.append((new_state, cost + 1))
        
        beam = sorted(next_beam, key=lambda x: x[1])[:beam_width]
    
    return None

# Ejecutar el algoritmo de beam search y medir el tiempo
beam_width = 3
start_time = time.time()
solution = beam_search(beam_width)
end_time = time.time()

# Imprimir la solución y el tiempo de ejecución
print("Beam Search Solution:")
if solution is None:
    print("No se encontró una solución válida.")
else:
    for exam, day in solution.items():
        print(f"{exam}: {day}")
    print(f"\nTiempo de ejecución: {end_time - start_time:.5f} segundos")

Beam Search Solution:
No se encontró una solución válida.


In [11]:
import time
import random

# Función para generar una asignación inicial aleatoria
def generate_initial_assignment():
    assignment = {}
    for exam in exams:
        assignment[exam] = random.choice(days)
    return assignment

# Función para obtener los vecinos de una asignación
def get_neighbors(assignment):
    neighbors = []
    for exam in exams:
        for day in days:
            if assignment[exam] != day:
                new_assignment = assignment.copy()
                new_assignment[exam] = day
                neighbors.append(new_assignment)
    return neighbors

# Función de local search
def local_search(max_iterations):
    current_assignment = generate_initial_assignment()
    
    for _ in range(max_iterations):
        if is_valid_assignment(current_assignment):
            return current_assignment
        
        neighbors = get_neighbors(current_assignment)
        best_neighbor = max(neighbors, key=lambda x: sum(1 for exam in exams if x[exam] != current_assignment[exam]))
        
        if is_valid_assignment(best_neighbor):
            current_assignment = best_neighbor
        else:
            break
    
    return None

# Ejecutar el algoritmo de local search y medir el tiempo
max_iterations = 1000
start_time = time.time()
solution = local_search(max_iterations)
end_time = time.time()

# Imprimir la solución y el tiempo de ejecución
print("Local Search Solution:")
if solution is None:
    print("No se encontró una solución válida.")
else:
    for exam, day in solution.items():
        print(f"{exam}: {day}")
    print(f"\nTiempo de ejecución: {end_time - start_time:.5f} segundos")

Local Search Solution:
No se encontró una solución válida.


In [12]:
import time
from prettytable import PrettyTable

# Ejecutar el algoritmo de backtracking search y medir el tiempo
start_time_backtracking = time.time()
solution_backtracking = backtracking_search()
end_time_backtracking = time.time()
time_backtracking = end_time_backtracking - start_time_backtracking

# Ejecutar el algoritmo de beam search y medir el tiempo
beam_width = 3
start_time_beam = time.time()
solution_beam = beam_search(beam_width)
end_time_beam = time.time()
time_beam = end_time_beam - start_time_beam

# Ejecutar el algoritmo de local search y medir el tiempo
max_iterations = 1000
start_time_local = time.time()
solution_local = local_search(max_iterations)
end_time_local = time.time()
time_local = end_time_local - start_time_local

# Crear una tabla para mostrar los resultados
table = PrettyTable()
table.field_names = ["Algoritmo", "Tiempo de ejecución (s)", "Solución encontrada"]

# Agregar los resultados a la tabla
table.add_row(["Backtracking Search", f"{time_backtracking:.5f}", "Sí" if solution_backtracking else "No"])
table.add_row(["Beam Search", f"{time_beam:.5f}", "Sí" if solution_beam else "No"])
table.add_row(["Local Search", f"{time_local:.5f}", "Sí" if solution_local else "No"])

# Imprimir la tabla de resultados
print("Comparación de algoritmos:")
print(table)

# Imprimir las soluciones encontradas por cada algoritmo
print("\nSoluciones encontradas:")

print("Backtracking Search:")
if solution_backtracking is None:
    print("No se encontró una solución válida.")
else:
    for exam, day in solution_backtracking.items():
        print(f"{exam}: {day}")

print("\nBeam Search:")
if solution_beam is None:
    print("No se encontró una solución válida.")
else:
    for exam, day in solution_beam.items():
        print(f"{exam}: {day}")

print("\nLocal Search:")
if solution_local is None:
    print("No se encontró una solución válida.")
else:
    for exam, day in solution_local.items():
        print(f"{exam}: {day}")

Comparación de algoritmos:
+---------------------+-------------------------+---------------------+
|      Algoritmo      | Tiempo de ejecución (s) | Solución encontrada |
+---------------------+-------------------------+---------------------+
| Backtracking Search |         0.00000         |          No         |
|     Beam Search     |         0.00052         |          No         |
|     Local Search    |         0.00000         |          No         |
+---------------------+-------------------------+---------------------+

Soluciones encontradas:
Backtracking Search:
No se encontró una solución válida.

Beam Search:
No se encontró una solución válida.

Local Search:
No se encontró una solución válida.
